In [ ]:
!pip install -q openai

In [ ]:
from pprint import pprint
from pydantic import BaseModel

def print_response(response):
  print(f"Response id: {response.id}")
  pprint(response.output)

  print()
  print(f"{'-' * 20} [Text] {'-' * 20}")
  print(response.output_text)

In [ ]:
prompts = {
    "business_naming": "Give me top 3 ideas for naming my business. It's a software company that innovates in audio recording.",
    "describe_ai": "Describe AI as if it were an animal.",
    "essay_about_humanity": "Write a short essay (up to 5 sentences) about the history of mankind.",
    "generate_character": "Generate a fictional character of an adorable but nerdy teenager."
}

## Basic usage

In [ ]:
from google.colab import userdata
from openai import OpenAI

api_key = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

In [ ]:
simple_response = client.responses.create(
    model="gpt-5-nano",
    input=prompts["essay_about_humanity"]
)

In [ ]:
print_response(simple_response)

## Reasoning

In [ ]:
low_reasoning_response = client.responses.create(
    model="gpt-5-nano",
    input=prompts["business_naming"],
    reasoning={"effort": "low"}
)

In [ ]:
print_response(low_reasoning_response)

In [ ]:
high_reasoning_response = client.responses.create(
    model="gpt-5-nano",
    input=prompts["business_naming"],
    reasoning={"effort": "high"}
)

In [ ]:
print_response(high_reasoning_response)

## Temperature

The GPT-5 models no longer support `temperature` so the next demonstrations will use an older model.

In [ ]:
deterministic_response = client.responses.create(
    model="gpt-4o-mini",
    input=prompts["describe_ai"],
    temperature=0.1
)

In [ ]:
print_response(deterministic_response)

In [ ]:
creative_response = client.responses.create(
    model="gpt-4o-mini",
    input=prompts["describe_ai"],
    temperature=0.9
)

In [ ]:
print_response(creative_response)

## Multi-turn conversations

This example includes `instructions` giving the model high-level guidance on how it should behave while generating a response.

> **OpenAI:**
> _"Note that the `instructions` parameter only applies to the current response generation request. If you are managing conversation state with the `previous_response_id` parameter, the `instructions` used on previous turns will not be present in the context."_



In [ ]:
first_turn_response = client.responses.create(
    model="gpt-5-nano",
    input="Generate a short role-play in no more than 5 sentences: manager and employee discussing performance issues.",
    instructions="The manager should talk like a pirate."
)

In [ ]:
print_response(first_turn_response)

In [ ]:
second_turn_response = client.responses.create(
    model="gpt-5-nano",
    input="Keep the role-play going for another 7 sentences, but introduce a new development: the employee's wife enters the room dramatically and tells him that she's pregnant, leaving him genuinely surpirsed but at the same time desperately hoping to save his job because his family depends on it.",
    previous_response_id=first_turn_response.id
)

In [ ]:
print_response(second_turn_response)

## Structured output

In [ ]:
no_structure_response = client.responses.create(
    model="gpt-5-nano",
    input=prompts["generate_character"]
)

In [ ]:
print_response(no_structure_response)

`Structured output` evolves from `JSON mode`, however it is no longer recommended to use the latter for a number of reasons.

> **OpenAI:**
> _When using JSON mode, you must always instruct the model to produce JSON via some message in the conversation, for example via your system message. If you don't include an explicit instruction to generate JSON, the model may generate an unending stream of whitespace and the request may run continually until it reaches the token limit._

> **OpenAI:**
> _JSON mode will not guarantee the output matches any specific schema, only that it is valid and parses without errors._

In [ ]:
diy_structured_response = client.responses.create(
    model="gpt-5-nano",
    input=prompts["generate_character"],
    text={
        "format": {
            "type": "json_schema",
            "name": "character",
            "schema": {
              "type": "object",
              "properties": {
                  "name": {"type": "string", "description": "The character's full name"},
                  "age": {"type": "integer", "description": "The character's age"},
                  "hobby": {"type": "string", "description": "The character's hobby"}
              },
              "required": ["name", "age", "hobby"],
              "additionalProperties": False,
            },
            "strict": True
        }
    }
)

In [ ]:
print_response(diy_structured_response)

In [ ]:
class Character(BaseModel):
    name: str
    age: int
    hobby: str

auto_structured_response = client.responses.parse(
    model="gpt-5-nano",
    input=prompts["generate_character"],
    text_format=Character
)



In [ ]:
print_response(auto_structured_response)

print()
print(f"Parsed output: {auto_structured_response.output_parsed}")

## Conversations API

While `responses` are saved for 30 days only, `conversations` are stored until deleted. [OpenAI: Data retention](https://developers.openai.com/api/docs/guides/your-data#storage-requirements-and-retention-controls-per-endpoint)

In [ ]:
conversation = client.conversations.create()
first_conversation_item = client.responses.create(
    model="gpt-5-nano",
    input="Which cuisine do you prefer—Italian or French?",
    conversation=conversation.id
)
second_conversation_item = client.responses.create(
    model="gpt-5-nano",
    input="And what do you think about the languages?",
    conversation=conversation.id
)

In [ ]:
print(f"Conversation id: {conversation.id}")

In [ ]:
print_response(first_conversation_item)

In [ ]:
print_response(second_conversation_item)

In [ ]:
all_conversation_items = client.conversations.items.list(conversation_id=conversation.id)
pprint(all_conversation_items.data)

## Vision

<img src="https://freerangestock.com/sample/88947/painter-working-in-studio.jpg" />


In [ ]:
image_analysis_response = client.responses.create(
    model="gpt-5-nano",
    input=[
        {
            "role": "system",
            "content": "You are an expert image analyst. Keep your answer concise and structured."
        },
        {
            "role": "user",
            "content": [
                { "type": "input_text", "text": "Analyze this image and return a one-sentence summary followed by 5 key visible objects." },
                { "type": "input_image", "image_url": "https://freerangestock.com/sample/88947/painter-working-in-studio.jpg" }
            ]
        }
    ]
)

In [ ]:
print_response(image_analysis_response)